In [ ]:
!source $HOME/physics/bin/activate

In [2]:
import sys
sys.path.append('../..')
from utils.rootLoad import rootLoad
rootLoad()

Shared objects loaded.
Now you can use ROOT in this notebook.


In [2]:
from ROOT import TFile, TH1D, TCanvas, TLegend, TLatex, TF1, gStyle, TGraphErrors, TMultiGraph
import uproot
import pandas as pd
import yaml


Welcome to JupyROOT 6.28/06


In [3]:
inTree = uproot.open('../Data/run175174828_230428174901_preprocessed.root')['PreprocessedData']
inPixels = [inTree[f'pixel{i}'].arrays(library='pd') for i in range(16)]
innerPixels = [inTree[f'pixel{i}'].arrays(library='pd') for i in [5, 6, 9, 10]]

In [4]:
outFile = TFile('../Data/output/spectrumVisual.root', 'recreate')
dirAmpl = outFile.mkdir('amplitude')
elecAmpl = outFile.mkdir('electrons')

In [5]:
def amplitudeSpectrumAndFits(inData, outDir, logFile, ipixel):

    gStyle.SetOptStat(0)

    h = TH1D(f'h_{ipixel}', 'h', 100, 1, 101)
    for x in inData['amplitude']:   h.Fill(x)
    h.SetFillColorAlpha(797, 0.5)
    h.SetLineColor(797)
    h.SetFillStyle(3356)
    h.SetTitle(f'Pixel {ipixel}; Signal amplitude (mV); Counts')

    with open(logFile, 'r') as f:   log = yaml.load(f, Loader=yaml.FullLoader)

    kAfit = TF1(f'kAfit_{ipixel}', '[0] * exp(-(x-[1])*(x-[1])/(2*[2]*[2]))', log[f'pixel{ipixel}']['fits'][0]['range'][0], 
                                                                    log[f'pixel{ipixel}']['fits'][0]['range'][1])
    kAfit.SetParameters(log[f'pixel{ipixel}']['fits'][0]['param'][0], 
                        log[f'pixel{ipixel}']['fits'][0]['param'][1], 
                        log[f'pixel{ipixel}']['fits'][0]['param'][2])
    kAfit.SetLineColor(860)
    kAfit.SetLineWidth(2)

    kBfit = TF1(f'kBfit_{ipixel}', '[0] * exp(-(x-[1])*(x-[1])/(2*[2]*[2]))', log[f'pixel{ipixel}']['fits'][1]['range'][0], 
                                                                    log[f'pixel{ipixel}']['fits'][1]['range'][1])
    kBfit.SetParameters(log[f'pixel{ipixel}']['fits'][1]['param'][0],
                        log[f'pixel{ipixel}']['fits'][1]['param'][1],
                        log[f'pixel{ipixel}']['fits'][1]['param'][2])
    kBfit.SetLineColor(419)
    kBfit.SetLineWidth(2)

    h.Fit(kAfit, 'RM+')
    print(f'Pixel {ipixel} kAfit: {kAfit.GetChisquare()} / {kAfit.GetNDF()}')
    h.Fit(kBfit, 'RM+')
    print(f'Pixel {ipixel} kBfit: {kBfit.GetChisquare()} / {kBfit.GetNDF()}')

    outDir.cd() 
    h.Write()
    kAfit.Write()
    kBfit.Write()

    return h, kAfit, kBfit


In [6]:
spectra = []
kAfits = []
kBfits = []

for ipixel, pixel in zip([5, 6, 9, 10], innerPixels):
    h, kAfit, kBfit = amplitudeSpectrumAndFits(pixel, dirAmpl, f'../src/config/simpleFitCfg.yml', ipixel)
    spectra.append(h)
    kAfits.append(kAfit)
    kBfits.append(kBfit)

Pixel 5 kAfit: 19.095126952648403 / 2
Pixel 5 kBfit: 31.644886991425473 / 3
Pixel 6 kAfit: 12.172128693090807 / 2
Pixel 6 kBfit: 27.095209023473544 / 2
Pixel 9 kAfit: 8.906797598715793 / 2
Pixel 9 kBfit: 31.623889667077588 / 3
Pixel 10 kAfit: 25.175636291299167 / 2
Pixel 10 kBfit: 28.99315904959549 / 3
 FCN=19.0951 FROM HESSE     STATUS=OK             16 CALLS          97 TOTAL
                     EDM=2.03848e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.20266e+03   3.10702e+01   5.27454e-02  -1.24840e-06
   2  p1           7.06358e+01   1.86923e-02   3.92187e-05  -3.13663e-03
   3  p2           8.42755e-01   1.57419e-02   2.58932e-05  -3.45795e-03
 FCN=31.6449 FROM HESSE     STATUS=OK             16 CALLS         118 TOTAL
                     EDM=6.6763e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER     

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [7]:
canvas = TCanvas('canvas', '^{55}Fe spectrum', 800, 600)
canvas.Divide(2, 2)
legends = []

for i, h in enumerate(spectra):
    frame = canvas.cd(i+1).DrawFrame(1, 0, 101, 1200, 'Pixel '+str(innerPixels[i]['pixel'][0])+'; Signal amplitude (mV); Counts')
    h.Draw('same')
    kAfits[i].Draw('same')
    kBfits[i].Draw('same')
    frame.GetYaxis().SetTitleSize(0.05)
    frame.GetYaxis().SetTitleOffset(1.)
    frame.GetXaxis().SetTitleSize(0.05)

    TLatex().SetTextSize(0.04)
    TLatex().SetTextFont(42)
    TLatex().DrawLatexNDC(0.2, 0.75, '^{55}Fe acquisition')
    TLatex().DrawLatexNDC(0.2, 0.7, 'K_{#alpha} = '+f'({kAfits[i].GetParameter(1):.1f} #pm {kAfits[i].GetParameter(2):.1f}) mV')
    TLatex().DrawLatexNDC(0.2, 0.65, 'K_{#beta} = '+f'({kBfits[i].GetParameter(1):.1f} #pm {kBfits[i].GetParameter(2):.1f}) mV')

    canvas.cd(i+1)
    legend = TLegend(0.2, 0.3, 0.5, 0.6)
    legend.SetBorderSize(0)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.06)
    legend.SetTextFont(42)

    legend.AddEntry(h, 'Data', 'f')
    legend.AddEntry(kAfits[i], 'K_{#alpha} fit', 'l')
    legend.AddEntry(kBfits[i], 'K_{#beta} fit', 'l')
    
    legends.append(legend)
    legends[i].Draw()


canvas.SaveAs('../Data/output/spectrumVisual.pdf')

Info in <TCanvas::Print>: pdf file ../Data/output/spectrumVisual.pdf has been created


In [8]:
canvas = TCanvas('canvas', 'Calibration curve', 800, 600)

graphs = []
fits = []
legends = []
colors = [860, 419, 797, 633]

inFile = TFile.Open('../Data/output/calibrationOutput.root')

for i, ipixel in enumerate([5, 6, 9, 10]):

    graph = inFile.Get(f'calibrationPx{ipixel}')
    fit = graph.GetFunction(f'calibrationPx{ipixel}Fit')
    #fit = inFile.Get(f'calibrationPx{ipixel}Fit')

    graph.SetMarkerStyle(20)
    #graph.SetMarkerSize(1.)
    graph.SetMarkerColor(colors[i]-1)
    graph.SetTitle(f'Pixel {innerPixels[i]["pixel"][0]}; Energy (keV); Amplitude (mV)')
    graph.SetLineColor(colors[i]-1)

    fit.SetLineColor(colors[i])
    fit.SetLineWidth(2)

    graphs.append(graph)
    fits.append(fit)

canvas.cd()

mg = TMultiGraph()
mg.SetTitle('Energy calibration; Energy (keV); Amplitude (mV)')
for graph in graphs:    mg.Add(graph)
mg.Draw('AP')
for fit in fits:    fit.Draw('same')

legend = TLegend(0.2, 0.5, 0.5, 0.75)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.04)
legend.SetTextFont(42)

for i, graph in enumerate(graphs):    legend.AddEntry(graph, f'Pixel {innerPixels[i]["pixel"][0]}', 'pl')

legend.Draw('same')

canvas.SaveAs('../Data/output/calibrationVisual.pdf')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../Data/output/calibrationVisual.pdf has been created


In [9]:
canvas = TCanvas('canvas', 'Calibration curve', 800, 600)

graphs = []
fits = []
legends = []
colors = [860, 419, 797, 633]

inFile = TFile.Open('../Data/output/calibrationOutput.root')

for i, ipixel in enumerate([5, 6, 9, 10]):

    graph = inFile.Get(f'calibrationPx{ipixel}Electrons')
    fit = graph.GetFunction(f'calibrationPx{ipixel}FitElectrons')
    #fit = inFile.Get(f'calibrationPx{ipixel}Fit')

    graph.SetMarkerStyle(20)
    #graph.SetMarkerSize(1.)
    graph.SetMarkerColor(colors[i]-1)
    graph.SetTitle(f'Pixel {innerPixels[i]["pixel"][0]}; Charge (e); Amplitude (mV)')
    graph.SetLineColor(colors[i]-1)

    fit.SetLineColor(colors[i])
    fit.SetLineWidth(2)

    graphs.append(graph)
    fits.append(fit)

canvas.cd()

mg = TMultiGraph()
mg.SetTitle('Charge calibration; Charge (e); Amplitude (mV)')
for graph in graphs:    mg.Add(graph)
mg.Draw('AP')
for fit in fits:    fit.Draw('same')

legend = TLegend(0.2, 0.5, 0.5, 0.75)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.04)
legend.SetTextFont(42)

for i, graph in enumerate(graphs):    legend.AddEntry(graph, f'Pixel {innerPixels[i]["pixel"][0]}', 'pl')

legend.Draw('same')

canvas.SaveAs('../Data/output/calibrationChargeVisual.pdf')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../Data/output/calibrationChargeVisual.pdf has been created


In [23]:
inFile = TFile.Open('../Data/output/spectralAnalysisOutput.root')
gStyle.SetOptStat(0)
gStyle.SetOptFit(0)

canvas = TCanvas('canvas', '', 800, 600)
spectrum = inFile.Get('spectrumSeedFull')

spectrum.SetTitle('^{55}Fe spectrum - seed; Seed Amplitude (e); Counts')
spectrum.SetLineColor(797)
spectrum.SetFillColorAlpha(797, 0.5)
spectrum.SetFillStyle(3356)

canvas.cd()
spectrum.Draw('hist')

latex = TLatex()
latex.SetTextSize(0.04)
latex.SetTextFont(42)
latex.DrawLatexNDC(0.2, 0.8, '^{55}Fe acquisition')


canvas.SaveAs('../Data/output/spectrumSeedFullVisual.pdf')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../Data/output/spectrumSeedFullVisual.pdf has been created


In [24]:
inFile = TFile.Open('../Data/output/spectralAnalysisOutput.root')
gStyle.SetOptStat(0)
gStyle.SetOptFit(0)

canvas = TCanvas('canvas', '', 800, 600)
spectrum = inFile.Get('spectrumSeed')

spectrum.SetTitle('^{55}Fe spectrum - seed; Seed Amplitude (e); Counts')
spectrum.SetLineColor(797)
spectrum.SetFillColorAlpha(797, 0.5)
spectrum.SetFillStyle(3356)

SiEscFit = spectrum.GetFunction('Si-escape')
SiEscFit.SetLineColor(633)

kAfit = spectrum.GetFunction('K-alpha')
kAfit.SetLineColor(860)

kBfit = spectrum.GetFunction('K-beta')
kBfit.SetLineColor(419)

legend = TLegend(0.2, 0.32, 0.5, 0.57)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.04)
legend.SetTextFont(42)
legend.AddEntry(spectrum, 'Data', 'f')
legend.AddEntry(SiEscFit, 'Si-escape', 'l')
legend.AddEntry(kAfit, 'K_{#alpha}', 'l')
legend.AddEntry(kBfit, 'K_{#beta}', 'l')

canvas.cd()
spectrum.Draw('hist')
kAfit.Draw('same')
kBfit.Draw('same')
SiEscFit.Draw('same')

legend.Draw('same')

latex = TLatex()
latex.SetTextSize(0.04)
latex.SetTextFont(42)
latex.DrawLatexNDC(0.2, 0.8, '^{55}Fe acquisition')
latex.DrawLatexNDC(0.2, 0.75, 'Fall time < 200 ps and cluster size = 1')    
latex.DrawLatexNDC(0.2, 0.7, 'Si-escape = '+f'({SiEscFit.GetParameter(1)*3.6/1000:.2f} #pm {SiEscFit.GetParameter(2)*3.6/1000:.2f}) keV')
latex.DrawLatexNDC(0.2, 0.65, 'K_{#alpha} = '+f'({kAfit.GetParameter(1)*3.6/1000:.2f} #pm {kAfit.GetParameter(2)*3.6/1000:.2f}) keV')
latex.DrawLatexNDC(0.2, 0.6, 'K_{#beta} = '+f'({kBfit.GetParameter(1)*3.6/1000:.2f} #pm {kBfit.GetParameter(2)*3.6/1000:.2f}) keV')


canvas.SaveAs('../Data/output/spectrumSeedVisual.pdf')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../Data/output/spectrumSeedVisual.pdf has been created


In [25]:
inFile = TFile.Open('../Data/output/spectralAnalysisOutput.root')
gStyle.SetOptStat(0)
gStyle.SetOptFit(0)

canvas = TCanvas('canvas', '', 800, 600)
spectrum = inFile.Get('spectrumCluster')

spectrum.SetTitle('^{55}Fe spectrum - cluster; Cluster Amplitude (e); Counts')
spectrum.SetLineColor(797)
spectrum.SetFillColorAlpha(797, 0.5)
spectrum.SetFillStyle(3356)

SiEscFit = spectrum.GetFunction('Si-escape')
SiEscFit.SetLineColor(633)

kAfit = spectrum.GetFunction('K-alpha')
kAfit.SetLineColor(860)

kBfit = spectrum.GetFunction('K-beta')
kBfit.SetLineColor(419)

legend = TLegend(0.2, 0.32, 0.5, 0.57)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.SetTextSize(0.04)
legend.SetTextFont(42)
legend.AddEntry(spectrum, 'Data', 'f')
legend.AddEntry(SiEscFit, 'Si-escape', 'l')
legend.AddEntry(kAfit, 'K_{#alpha}', 'l')
legend.AddEntry(kBfit, 'K_{#beta}', 'l')

canvas.cd()
spectrum.Draw('hist')
kAfit.Draw('same')
kBfit.Draw('same')
SiEscFit.Draw('same')

legend.Draw('same')

latex = TLatex()
latex.SetTextSize(0.04)
latex.SetTextFont(42)
latex.DrawLatexNDC(0.2, 0.8, '^{55}Fe acquisition')
latex.DrawLatexNDC(0.2, 0.75, 'Fall time < 200 ps and cluster size <= 2')
latex.DrawLatexNDC(0.2, 0.7, 'Si-escape = '+f'({SiEscFit.GetParameter(1)*3.6/1000:.2f} #pm {SiEscFit.GetParameter(2)*3.6/1000:.2f}) keV')
latex.DrawLatexNDC(0.2, 0.65, 'K_{#alpha} = '+f'({kAfit.GetParameter(1)*3.6/1000:.2f} #pm {kAfit.GetParameter(2)*3.6/1000:.2f}) keV')
latex.DrawLatexNDC(0.2, 0.6, 'K_{#beta} = '+f'({kBfit.GetParameter(1)*3.6/1000:.2f} #pm {kBfit.GetParameter(2)*3.6/1000:.2f}) keV')


canvas.SaveAs('../Data/output/spectrumClusterVisual.pdf')

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file ../Data/output/spectrumClusterVisual.pdf has been created
